# SPARQL: Die RDF Abfragesprache

In dieser Übung schauen wir uns die SPARQL Abfragesprache für RDF in der Praxis an. Wir verwenden dafür etwas RDF mit der Pink Floyd Diskographie und führen einige Abfragen aus. Zum Schluss schreiben Sie dann Ihr eigenes RDF und einige SPARQL Abfragen dazu.

In [1]:
!pip install rdflib

In [2]:
import pandas as pd
from io import BytesIO, StringIO
from rdflib import Graph
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer
from IPython.display import display

rdf = """
@prefix ex: <http://example.org#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a ex:Album ;
   ex:title "The Dark Side of the Moon"^^xsd:string ;
   ex:label "Harvest, EMI"@en ;
   ex:released [ 
     ex:day "16"^^xsd:int ;
     ex:month "03"^^xsd:int ;
     ex:year "1973"^^xsd:int 
   ] .
   
[] a ex:Album ;
   ex:title "The Wall" ;
   ex:label "Harvest, EMI" ;
   ex:released [ 
     ex:day 30 ;
     ex:month "11"^^xsd:string ;
     ex:year "1979"^^xsd:int 
   ] .

[] a ex:Single ;
   ex:title "What God Wants, Part 1"^^xsd:string ;
   ex:author [
     ex:firstname "Roger" ;
     ex:lastname "Waters"
   ] ;
   ex:released [ 
     ex:year "1992"^^xsd:int 
   ] .
"""

g = Graph()

r = g.parse(data=rdf, format='turtle')

def query(q):
    serializer = CSVResultSerializer(g.query(q))
    output = BytesIO()
    serializer.serialize(output)
    display(pd.read_csv(StringIO(output.getvalue().decode())))

Führen Sie nun die folgenden Abfragen aus und beantworten Sie die Fragen.

In [3]:
query("""
SELECT ?title WHERE { 
  ?work ex:title ?title
}
""")

,title
0,The Wall
1,The Dark Side of the Moon
2,"What God Wants, Part 1"


In [4]:
query("""
SELECT ?work ?title WHERE { 
  ?work ex:title ?title
}
""")

,work,title
0,ub1bL14C1,The Wall
1,ub1bL5C1,The Dark Side of the Moon
2,ub1bL23C1,"What God Wants, Part 1"


In [5]:
query("""
SELECT ?title WHERE { 
  [] ex:title ?title
}
""")

# Was ist der Unterschied zum vorherigem Beispiel? Antwort: der Spaltenname work taucht nicht auf, da er in der Abfrage auch nicht auftaucht. Dadurch wird auch kein Listenpunkt "work" erstellt.

,title
0,The Wall
1,The Dark Side of the Moon
2,"What God Wants, Part 1"


In [6]:
query("""
SELECT ?title WHERE { 
  [] rdf:type ex:Album ; 
     ex:title ?title
}
""")

# Warum nur zwei Resultate? Antwort: weil "What God Wants, Part 1" den Präix ex:single hat. Hier wird explizit nach rdf:type:ex:album gefragt.

,title
0,The Wall
1,The Dark Side of the Moon


In [7]:
query("""
SELECT ?s ?p ?o WHERE { 
  ?s ?p ?o
}
""")

# Was erhält man hier? Antwort: da im Tripelmuster nur Variablen auftauchen, können alle Elemente aus dem rdf Dokument beliebig eingesetzt werden. ALso für ?s kann zb jedes Subjekt eingesetzt werden

,s,p,o
0,ub1bL14C1,http://example.org#label,"Harvest, EMI"
1,ub1bL25C14,http://example.org#lastname,Waters
2,ub1bL14C1,http://example.org#title,The Wall
3,ub1bL23C1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org#Single
4,ub1bL5C1,http://example.org#label,"Harvest, EMI"
5,ub1bL5C1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://example.org#Album
6,ub1bL8C16,http://example.org#day,16
7,ub1bL23C1,http://example.org#author,ub1bL25C14
8,ub1bL23C1,http://example.org#released,ub1bL29C16
9,ub1bL23C1,http://example.org#title,"What God Wants, Part 1"


In [8]:
query("""
SELECT ?title ?year WHERE { 
  [] rdf:type ex:Album ; 
     ex:title ?title ;
     ex:released [ ex:year ?year ]
  FILTER (?year > 1973)
}
""")

,title,year
0,The Wall,1979


In [9]:
query("""
SELECT ?title ?year WHERE { 
  {
    [] rdf:type ex:Album ; 
       ex:title ?title ;
       ex:released [ ex:year ?year ]
    FILTER (?year > 1973)
  }
  UNION
  {
    [] rdf:type ex:Single ; 
       ex:title ?title ;
       ex:released [ ex:year ?year ]
    FILTER (?year <= 2000 )
  }
}
""")

,title,year
0,The Wall,1979
1,"What God Wants, Part 1",1992


In [10]:
query("""
SELECT ?title ?label WHERE { 
    ?work ex:title ?title .
    OPTIONAL { ?work ex:label ?label }
}
""")

# Warum ist der label von "What God Wants, Part 1" NaN? Antwort: da kein label hinterlegt ist. 

,title,label
0,The Wall,"Harvest, EMI"
1,The Dark Side of the Moon,"Harvest, EMI"
2,"What God Wants, Part 1",NaN


In [11]:
query("""
SELECT ?title WHERE { 
  [] rdf:type ex:Album ;
     ex:title ?title ;
     ex:label ?label
  FILTER (LANG(?label) = "en")
}
""")

,title
0,The Dark Side of the Moon


In [12]:
query("""
SELECT ?title WHERE { 
  [] rdf:type ex:Album ;
     ex:title ?title ;
     ex:released [ ex:day ?day ]
  FILTER (?day > 15)
}
""")

,title
0,The Wall
1,The Dark Side of the Moon


In [13]:
query("""
SELECT ?title WHERE { 
  [] rdf:type ex:Album ;
     ex:title ?title ;
     ex:released [ ex:month ?month ]
  FILTER (DATATYPE(?month) = xsd:string)
}
""")

# Warum erhält man hier nur "The Wall" als Resultat? Antwort: da der Datentyp von month bei "The Dark Side of the Moon" ein integer ist. Die single wird nicht abgefragt, weil explizit alben abgefragt werden.

,title
0,The Wall


Schreiben Sie nun ihr eigenes RDF und werten Sie einige SPARQL Abfragen aus.

In [26]:
rdf = """
@prefix ex: <http://example.org#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:buch rdf:type ex:buecher ;
    ex:title "Das leere Haus"^^xsd:string ; 
    ex:author "Blackwood"^^xsd:string; .

"""

g = Graph()

r = g.parse(data=rdf, format='turtle')

In [31]:
query("""
SELECT ?title ?author WHERE { 
    ?buch rdf:type ex:buecher ;
    ex:title ?title ;
    ex:author ?author.
    
}
""")

,title,author
0,Das leere Haus,Blackwood
